<h1 style="text-align:center;"><strong><em>MERCADO LIBRE SCRAPER</em></strong></h1>


#### Enter between ' ' the item on which you want to obtain the data in Mercado Libre.
#### You will receive a csv file with that name with all the information from your search.

In [81]:
item = 'posters del 89 de Union' 

In [82]:
item

'posters del 89 de Union'

In [83]:
import csv
import requests
from bs4 import BeautifulSoup
from lxml import etree
import re
import os

# Initial URL for web scraping
siguiente = f'https://listado.mercadolibre.com.ar/{item}'


# Lists to store the extracted data
lista_titulos = []
lista_url = []
lista_precios = []

# Loop to iterate over all result pages
while True:
    # Perform an HTTP GET request to the URL
    r = requests.get(siguiente)
    # Check if the request was successful

    if r.status_code == 200:
        # Create a BeautifulSoup object from the response content
        soup = BeautifulSoup(r.content, 'html.parser')

        # Create a DOM object from the response content
        dom = etree.HTML(str(soup))

        # Find all h2 elements with the specified class and extract the text from each one
        titulos = soup.find_all('h2', attrs={"class": "ui-search-item__title shops__item-title"})

        # Replace all commas with spaces in each title
        titulos = [i.text.replace(',', ' ') for i in titulos]

        # Add the titulos to the list of titulos
        lista_titulos.extend(titulos)

        # Find all elements 'a' with the specified class and extract the value of the 'href' attribute from each one
        urls = soup.find_all('a', attrs={"class": "ui-search-item__group__element shops__items-group-details ui-search-link"})
        urls = [i.get('href') for i in urls]

        # Add the URLs to the list of URLs
        lista_url.extend(urls)
        
        # Find all elements 'div' with the specified class and extract the text from each one
        precio = soup.find_all('div', attrs={"class": "ui-search-price__second-line shops__price-second-line"})

        # Use a regular expression to extract only digits and the decimal point from each price text
        precio = [re.search(r'\d+\.\d+', i.text).group() if re.search(r'\d+\.\d+', i.text) else '' for i in precio]
        
        # Add the prices to the list of prices
        lista_precios.extend(precio)
        
        # Find the 'span' element with the specified class and extract the text if it exists
        botoninicial_element = soup.find('span', attrs={"class": "andes-pagination__link"})
        if botoninicial_element:
            botoninicial = botoninicial_element.text
            # Convert the text to an integer
            botoninicial = int(botoninicial)
        else:
            # Handle the case where the element is not found (e.g., set a default value or break out of the loop)
            botoninicial = 0  # You can set a default value or handle this as needed

        # Find the 'li' element with the specified class and extract the text if it exists
        can_element = soup.find('li', attrs={"class": "andes-pagination__page-count"})
        if can_element:
            # Split the text into words and take the second word (the total number of pages)
            can = int(can_element.text.split(" ")[1])
        else:
            # Handle the case where the element is not found (e.g., set a default value or break out of the loop)
            can = 0  # You can set a default value or handle this as needed

        # Check if we have reached the last page
        if botoninicial == can:
            break

        # Use XPath to find the 'a' element that represents the "Next" button and extract the value of the href attribute
        next_button_elements = dom.xpath('//div[@class="ui-search-pagination shops__pagination-content"]//ul/li[contains(@class,"--next")]/a')

        # Check if there are any matching elements
        if next_button_elements:
            # If there are matching elements, extract the href attribute
            siguiente = next_button_elements[0].get('href')
        else:
            # If there are no matching elements, break out of the loop
            break

    else:
        # If the request was not successful, break out of the loop
        break

# Open a CSV file in write mode and UTF-8 encoding.
with open(f'{item}.csv', 'w', newline='', encoding='utf-8') as file:
    # Create a CSV writer with non-numeric quoting
    writer = csv.writer(file, quoting=csv.QUOTE_NONNUMERIC)  
    # Write the header row to the CSV file
    writer.writerow(['Titulo', 'Precio', 'URL'])

    # Iterate over the lists of titles, prices, and URLs and write each row to the CSV file
    for titulo, precio, url in zip(lista_titulos, lista_precios, lista_url):
        writer.writerow([titulo, precio, url])